<a href="https://colab.research.google.com/github/shilpasy/Building-and-Interpreting-Random-Forest-based-Cell-Penetrating-Peptide-Prediction-Model/blob/master/Feature_Generation_for_Protein_peptide_Sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple feature calculations for protein and peptide sequences to use in AI/ML related projects. These can be a basic set of features on which you can develop further depending on the problem of interest.

In [ ]:
#!pip install biopython

In [ ]:
import os
import random
import math
import numpy as np
import pandas as pd
import itertools
from collections import namedtuple
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis # you can install this library using pip
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
#warnings.filterwarnings('ignore')
#pd.set_option('display.max_rows', 1000)

Your input file i.e. training data which has sequences and target-labels is "train" here and the way in which you call the function is all_features_df = generate_features(train). This dataframe will have the calculated features. Features calculated here: amino acid frequencies, dipeptide frequencies, tripeptide frequencies, common biochemical properties

In [ ]:
def calculate_aa_frequencies(sequence, peptides):
    aa_frequencies = [sequence.count(aa) / len(sequence) for aa in peptides]
    return aa_frequencies

def generate_features(train):
    dipeptide = [''.join(x) for x in itertools.product('ACDEFGHIKLMNPQRSTVWY', repeat=2)]
    # tripeptide = [''.join(y) for y in itertools.product('ACDEFGHIKLMNPQRSTVWY', repeat=3)]
    biochem_features = ["net_charge", "IsoelectricPoint", "Hbdonors", "Hbacceptors", "HbDonorAcceptorDiff", "hydropathy", "MolWt", "SS_B", "SS_T", "SS_H"]

    features = []

    for i, row in train.iterrows():
        sequence = row['protein_sequence']
        labelclass = row['tm']

        aa_frequencies = calculate_aa_frequencies(sequence, 'ACDEFGHIKLMNPQRSTVWY')
        #dipeptide_count = calculate_aa_frequencies(sequence, dipeptide)
       # tripeptide_count = calculate_aa_frequencies(sequence, tripeptide)

        # Biochemical properties features
        eachSeq = sequence.replace('B', 'N').replace('Z', 'Q').replace('J', 'I').replace('X', 'G').replace('m', '')
        positive_aa_R = eachSeq.count('R')
        positive_aa_K = eachSeq.count('K')
        negative_aa_D = eachSeq.count('D')
        negative_aa_E = eachSeq.count('E')
        aromaticnum_W = eachSeq.count('W')
        aromaticnum_Y = eachSeq.count('Y')
        aromaticnum_F = eachSeq.count('F')
        count_aa_H = eachSeq.count('H'); count_aa_N = eachSeq.count('N'); count_aa_Q = eachSeq.count('Q'); count_aa_S = eachSeq.count('S'); count_aa_T = eachSeq.count('T');
        net_charge = (positive_aa_R + positive_aa_K - negative_aa_D - negative_aa_E) / len(eachSeq)
        PI = ProteinAnalysis(eachSeq).isoelectric_point()
        Hbdonors = (positive_aa_R * 5 + count_aa_Q * 2 + count_aa_S + count_aa_T + count_aa_H * 2 +
                    positive_aa_K * 3 + aromaticnum_W + aromaticnum_Y + count_aa_N * 2) / len(eachSeq)
        Hbacceptors = (count_aa_N * 2 + negative_aa_D * 4 + count_aa_Q * 2 + negative_aa_E * 4 +
                       count_aa_H * 2 + count_aa_S * 2 + count_aa_T * 2 + aromaticnum_Y) / len(eachSeq)
        Hbdonaccdiff = (Hbdonors - Hbacceptors) / len(eachSeq)
        SS = ProteinAnalysis(eachSeq).secondary_structure_fraction()  # [Sheet, Turn, Helix]

        hydropathy = ProteinAnalysis(eachSeq).gravy()
        MW = ProteinAnalysis(eachSeq).molecular_weight()
        MW = MW / len(eachSeq)

        # Combine all features
        #all_features = aa_frequencies + dipeptide_count + [ .... ]
        all_features = aa_frequencies + [net_charge, PI, Hbdonors, Hbacceptors, Hbdonaccdiff, hydropathy, MW, SS[0], SS[1], SS[2]]
        all_features.append(labelclass)
        features.append(all_features)
    #columns = ['aa_' + aa for aa in 'ACDEFGHIKLMNPQRSTVWY'] + ['dipeptide_' + dp for dp in dipeptide] + [f"biochem_{feature}" for feature in biochem_features] + ['LABEL'] # # ['tripeptide_' + tp for tp in tripeptide] +
    columns = ['aa_' + aa for aa in 'ACDEFGHIKLMNPQRSTVWY'] + [f"biochem_{feature}" for feature in biochem_features] + ['LABEL']

    features_df = pd.DataFrame(features, columns=columns)

    return features_df

###### Generate all features ########
all_features_df = generate_features(train)